<a href="https://colab.research.google.com/github/olgasem10/GPT_poetry/blob/main/GPT_finetuning_poetry.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install transformers

     |████████████████████████████████| 1.5MB 7.8MB/s 
     |████████████████████████████████| 2.9MB 14.8MB/s 
     |████████████████████████████████| 890kB 54.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=b3a03fbc247c0eded558294ea20ce1c28c5f1cb7859bd482f81e738bb44a33af
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [4]:
import torch
from transformers import AutoModelWithLMHead, AutoTokenizer, AdamW, get_linear_schedule_with_warmup

In [5]:
model = AutoModelWithLMHead.from_pretrained('sberbank-ai/rugpt3small_based_on_gpt2')

/usr/local/lib/python3.6/dist-packages/transformers/models/auto/modeling_auto.py:890: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


In [6]:
tokenizer = AutoTokenizer.from_pretrained('sberbank-ai/rugpt3small_based_on_gpt2')

Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.


##Подготовка данных

Данные для обучения - датасет из примерно 44400 стихотворений, для каждого стихотворения известен год написания, стихотворения длиной не более 100 строчек.
В качестве условия для генерации я решила использовать век написания каждого стихотворения, рассчитывая что в итоге стихи сгенерированные для разных веков будут выглядеть по-разному


In [ ]:
from torch.utils.data import Dataset, random_split, DataLoader, RandomSampler, SequentialSampler

In [37]:
import pandas as pd
from tqdm import tqdm
import numpy as np
import random
import json

In [13]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [35]:
home_dir = 'drive/My Drive/GPT/'

In [38]:
df = pd.read_csv(home_dir + 'Poems.csv').dropna()

In [ ]:
df.shape

(44402, 2)

In [39]:
def cent(year):
    return int(str(year)[:2]) +1

In [40]:
df['century'] = df['date'].apply(cent)

In [16]:
df = df.sample(frac=1).reset_index(drop=True)

In [18]:
df.head()

,poem,date,century
0,"Во мраке безрадостном ночи,\nДушевной больной ...",1903,20
1,"Тянется собранье, длятся пренья ...\nИ когда о...",1957,20
2,Сквозь зеленые буки желтеют чужие поля.\nЧереп...,1923,20
3,"Там, где картинно обгибая\nБрега, одетые в гра...",1831,19
4,"Хлеб от земли, а голод от людей:\nЗасеяли расс...",1923,20


In [ ]:
raw = df['poem'].to_list()
cents = df['century'].to_list()

In [ ]:
len(raw)

44402

In [ ]:
toks = [tokenizer.encode(text) for text in raw]

In [ ]:
validation_start_index = int(len(raw) * 0.1)
validation_start_index

4440

In [ ]:
train_lengths = [len(tok) for tok in tqdm(toks)]

100%|██████████| 44402/44402 [00:00<00:00, 2152339.57it/s]


In [ ]:
np.percentile(train_lengths, 95)

506.0

К сожалению при попытке создавать батчи из нескольких стихотворений происходила ошибка Cuda out of memory, поэтому пришлось обучаться по каждому стихотворению отдельно (т.е. batch_size = 1), поэтому паддинги я в итоге не использовала


In [ ]:
batch_size = 1
max_len = 500
pad_index = 50258
boc_index = 50259
eoc_index = 50260

In [7]:
tokenizer.add_special_tokens({'additional_special_tokens':['[PAD]', '[BOC]', '[EOC]']})

3

In [ ]:
class PoetryDataset(Dataset):
    
    def __init__(self, texts, cents, tokenizer, max_length):
        
        
        self.tokenizer = tokenizer
        self.texts = texts
        self.cents = cents

        self.max_len = max_length

    def __len__(self):        
        return len(self.texts)
    
    def __getitem__(self, index):
        
        text = self.texts[index]
        cent = self.cents[index]
        text = f'[BOC]{cent}[EOC]{text}'
        
        enc_dict = tokenizer(text, truncation=True, max_length=self.max_len)
        tokenized = enc_dict['input_ids']
        
        return torch.tensor(tokenized).long()

In [ ]:
poems_train = PoetryDataset(raw[:-validation_start_index], cents[:-validation_start_index], tokenizer, max_len)
poems_valid = PoetryDataset(raw[-validation_start_index:], cents[-validation_start_index:],tokenizer, max_len)

len(poems_train), len(poems_valid)

(39962, 4440)

In [ ]:
poems_train[33333]

tensor([50259,  2932, 50260,  4348,   281,  5557,   466,   306,  4144, 27895,
          203,   677, 22914,   680, 21696,    16,   203,  3693, 37938,   670,
        31280,   203,  1009,   826,   277,   290, 10453,   404,    18,   203,
         6039, 17921, 20549,  1224,   203,   563,  3583,   285,  9201,   581,
        13734,    18,   203,   618,   282, 14854,  6237, 10135, 37207,   282,
        10764,    16,   203,  3693, 20789,   830,  1604,   484, 23149,    18,
          203,  1098,  8371, 23678,  2356, 10657,   454,   305,   203,  2176,
        49949, 17055,   933, 43441,    16,   203, 21037,  1071,  3978,  6996,
          328,  1788,    16,   203,   677,   328,   303,   770,  6050,   282,
         3978,  4533,    18,   203, 20377,  5576,  2028,   770, 18673,    16,
          203, 24860,    16,   322, 38661,   417,  3069,   203, 49932,  1886,
          557,  3792,   520,   357,   406,   203,   684,  3822,   548, 39139,
          416,  6456,    31,   203,  4529,  1271, 25268,  1211, 

In [ ]:
train_loader = DataLoader(poems_train,
                          batch_size=batch_size,
                              shuffle=True)

validation_loader = DataLoader(poems_valid,
                            batch_size=batch_size,
                            shuffle=True)

### Обучение

In [10]:
device = 'cuda'

In [ ]:
num_epochs = 3

In [ ]:
learning_rate = 0.0001
warmup_steps = 50
total_steps = len(train_loader) * num_epochs

In [ ]:
optimizer = AdamW(model.parameters(), lr=learning_rate)

In [ ]:
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps=warmup_steps,
                                            num_training_steps=total_steps)

In [8]:
model.resize_token_embeddings(len(tokenizer))

Embedding(50261, 768)

In [11]:
model = model.to(device)

In [ ]:
def train(model, loader, optimizer, scheduler, last_n_losses=200, verbose=True):

    losses = []

    progress_bar = tqdm(total=len(loader.dataset), disable=not verbose, desc='Train')

    model.train()

    for toks in loader:

        toks = toks.to(device)
        
        outputs = model(toks,
                        labels=toks
                        )
        loss = outputs[0]
        losses.append(loss.item())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()

        progress_bar.set_postfix(loss=np.mean(losses[-last_n_losses:]),
                                 perplexity=np.exp(np.mean(losses[-last_n_losses:])))
        
        progress_bar.update()

    progress_bar.close()
    
    return losses

    

In [ ]:
def evaluate(model, loader, last_n_losses=200, verbose=True):

    losses = []

    progress_bar = tqdm(total=len(loader), disable=not verbose, desc='Evaluate')

    model.eval()

    for toks in loader:

        toks = toks.to(device)
        
        outputs = model(toks,
                        labels=toks
                        )
        loss = outputs[0]
        losses.append(loss.item())

        progress_bar.set_postfix(loss=np.mean(losses[-last_n_losses:]),
                                 perplexity=np.exp(np.mean(losses[-last_n_losses:])))
        
        progress_bar.update()

    progress_bar.close()
    
    return losses

In [ ]:
save_best_model_path = '/content/drive/My Drive/GPT/mod/best_model_state_dict.pth'
save_best_optimizer_path = '/content/drive/My Drive/GPT/mod/best_optimizer_state_dict.pth'
save_last_model_path = '/content/drive/My Drive/GPT/mod/last_model_state_dict.pth'
save_last_optimizer_path = '/content/drive/My Drive/GPT/mod/last_optimizer_state_dict.pth'

In [ ]:
#tqdm._instances.clear()

In [ ]:
train_losses = []
validation_losses = []

train_perplexities = []
validation_perplexities = []

best_validation_loss = 1e+6

for n_epoch in range(1, num_epochs + 1):
    
    epoch_train_losses = train(model, train_loader, optimizer, scheduler)
    epoch_validation_losses = evaluate(model, validation_loader)
    
    mean_train_loss = np.mean(epoch_train_losses)
    mean_validation_loss = np.mean(epoch_validation_losses)
    
    train_losses.extend(epoch_train_losses)
    train_perplexities.append(np.exp(mean_train_loss))
    
    validation_losses.extend(epoch_validation_losses)
    validation_perplexities.append(np.exp(mean_validation_loss))
    
    message = f'Epoch: {n_epoch}\n'
    message += f'Train: loss - {mean_train_loss:.4f} | perplexity - {train_perplexities[-1]:.3f}\n'
    message += f'Validation: loss - {mean_validation_loss:.4f} | perplexity - {validation_perplexities[-1]:.3f}'
    
    print(message)
    
    if mean_validation_loss < best_validation_loss:
        
        best_validation_loss = mean_validation_loss
        
        torch.save(model.state_dict(), save_best_model_path)
        torch.save(optimizer.state_dict(), save_best_optimizer_path)
        
    else:
        break
        
    torch.save(model.state_dict(), save_last_model_path)
    torch.save(optimizer.state_dict(), save_last_optimizer_path)

    with open(home_dir + f'info_{n_epoch}.json', 'w') as file_object:

        info = {
            'message': message,
            'train_losses': train_losses,
            'validation_losses': validation_losses,
            'train_perplexities': train_perplexities,
            'validation_perplexities': validation_perplexities
        }

        file_object.write(json.dumps(info, indent=2))

Evaluate: 100%|██████████| 4440/4440 [01:59<00:00, 37.03it/s, loss=3.81, perplexity=45.4]


Epoch: 1
Train: loss - 4.0246 | perplexity - 55.958
Validation: loss - 3.7932 | perplexity - 44.396


Evaluate: 100%|██████████| 4440/4440 [01:59<00:00, 37.16it/s, loss=3.66, perplexity=38.8]


Epoch: 2
Train: loss - 3.6460 | perplexity - 38.319
Validation: loss - 3.6657 | perplexity - 39.085


Evaluate: 100%|██████████| 4440/4440 [01:57<00:00, 37.68it/s, loss=3.71, perplexity=41]


Epoch: 3
Train: loss - 3.3325 | perplexity - 28.008
Validation: loss - 3.6475 | perplexity - 38.378


### Генерация

In [ ]:
model.load_state_dict(torch.load('/content/drive/My Drive/GPT/mod/best_model_state_dict.pth'))
optimizer.load_state_dict(torch.load('/content/drive/My Drive/GPT/mod/best_optimizer_state_dict.pth'))

In [15]:
def write_poem(century, snippet, length, tokenizer, model):
    input_text = f'[BOC]{century}[EOC]{snippet}'
    inds = tokenizer(input_text, return_tensors='pt')['input_ids'].to(device)
    text = model.generate(
        input_ids=inds,
        max_length=length + len(inds),
        do_sample=True,
        no_repeat_ngram_size=20,
        pad_token_id = 50258
        )
    poem = tokenizer.decode(text[0])
    poem = poem.strip(f'[BOC]{century}[EOC]')
    return poem
    

Пример генерации стихотворений с одним и тем же началом, но для разных веков:

In [18]:
poem = write_poem(18, 'Вечер пришел и стало темно', 100, tokenizer, model)
print(poem)

Вечер пришел и стало темно,
И над нами с облаков
Осенний свет вечерней порою
Уж светит в поле золотистом;
И о чем же ты, о небо,
Очи своих сияющих дней
Сияньем звезд своих кажешься
И в облаке, в облаке своем?
О, не то -- я слышу голос твой,
Твой голос в тишине ночной,
И с твоих небес ясных и чистых
Навсегда


In [19]:
poem = write_poem(19, 'Вечер пришел и стало темно', 100, tokenizer, model)
print(poem)

Вечер пришел и стало темно,
Узорь заката так черна.
Мы в дальнюю даль взглянули робко.
И вот нам на земле темно.
Кто там с улыбкою сидит?
Кто играет на ветке белой?
Кто, смеясь и плача над нами,
Говорит, грустя и плача?
И нам показалось, что давно
Смотрел в окно над водой,
Что здесь на земле темно.
Глядя в глаза


In [23]:
poem = write_poem(20, 'Вечер пришел и стало темно', 100, tokenizer, model)
print(poem)

Вечер пришел и стало темно,
И мы глядим, как в углу
Шуршит, как на листке, книжка,
Разворачивая лисьи свитки.
Какие звезды светят
В глазах у нас, у других!
Уж ночь опустилась низко,
Как в темень прошлого века.
В этом, пожалуй, не надо
Ни горя, ни счастья иного,
Ни смерти мучительной --
Пора бы уже спать!
Но этот


In [26]:
poem = write_poem(21, 'Вечер пришел и стало темно', 100, tokenizer, model)
print(poem)

Вечер пришел и стало темно:
В небе зной, и тишина,
И тихо качаюсь, как сна птица,
Качаемая ветром.
Это было, казалось, давно.
Тени прошлого так свежи,
Так прозрачны, что даже не знакомы,
Эти люди, что так близко к смерти.
Простившись, мы прошли до вокзала
И не вспомнили, зачем,
Стали думать: всё это было,
Вот он пройдет и


В итоге сложно проследить какие-то явные отличия между стихами для разных веков. Возможно отчасти это связано с тем, что соотношение количества стихов по векам в датасете было неравным (например, всего лишь 41 стихотворение для 21 века). К тому же в получившихся стихах почти никогда нет рифмы

In [43]:
df['century'].value_counts()

20    32344
19    11605
18      412
21       41
Name: century, dtype: int64

Интересно, однако, что в итоге модель при генерации умеет подстраиваться под длину строки (то есть генерирует строки примерно одинакового размера на протяжении всего стихотворения)

In [54]:
poem = write_poem(20, 'Вот дом, а вот окно, из которого я глядел когда-то', 100, tokenizer, model)
print(poem)

Вот дом, а вот окно, из которого я глядел когда-то на нее.
Здесь жили те, кто не были, а жили те, что когда-то я увидал ее и сейчас.
Потускнели от слез и печали ее яркие щеки,
Но глаза ее были как живые; так в ясном небе ясно ей,
Что в небе ясно виделась ей она: на груди ее я стоял и дрожал.
Я понял ее давно, но было поздно:


In [51]:
poem = write_poem(20, 'Вот дом\nА вот окно', 60, tokenizer, model)
print(poem)

Вот дом
А вот окно,
А вот и он,
И вот
Он весь в розах
И в садах
Двух деревьев,
Вдоль по мостам,
На двух парах,
На тройках,
Грустных,



В принципе в стихотворении поддерживается тема, заданная в начале (если ввести что-то про зиму, то в стихотворении может упоминаться снег, что-то про город - улицы и машины и так далее)

In [69]:
poem = write_poem(20, 'Среди лесов осенних и пустых', 100, tokenizer, model)
print(poem)

Среди лесов осенних и пустых,
В лесу, посреди полей пустых,
Напоминают мне поляны старые
Средь зеленистых ольш.
Над долиной, над долиной, над долиной
Чуть виден лес из облаков.
Незабвенный, родной и прекрасный,
Родной и близкий мне, как ты.
Лес, как будто мертвый, -- одинокий,
Глубже, чем на родине -- лес.



In [71]:
poem = write_poem(20, 'Не гаснет свет в твоем окне', 100, tokenizer, model)
print(poem)

Не гаснет свет в твоем окне.
Ты не спишь. Я смотрю туда,
Где, как призрак, ночь таится,
Вьется в моей руке.
Туда, где свет лежит в окне,
Кто не спит, чтоб не проснуться,
И не гаснет свет в твоем окне
На вечернем заре.
И еще во тьме, во мгле ночи
Увидел я твой огонь,
Что в глаза твой взор проникал
И вот


Еще отмечу, что почему-то почти все стихи получаются грустные